In [1]:
import pandas as pd

import protfasta
import requests
import re
import warnings
warnings.filterwarnings("ignore")
import glob


from Bio import AlignIO
import matplotlib.pyplot as plt

from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import rich
from rich_msa import RichAlignment

import logomaker

#Import modules
import gzip, logomaker, matplotlib.pyplot as plt, numpy as np, os, pandas as pd, protfasta


import seaborn as sns
import warnings
warnings.filterwarnings("ignore")


Matplotlib is building the font cache; this may take a moment.


# 1. Downloading codon alignments for all TFs

First, using uniprot to get all possible gene names.

In [7]:
lambert_TFs = pd.read_csv("../output/lambert_TFs_10-21-24_with_DBD_coords.csv", index_col = 0)
lambert_TFs["uniprotID"] = lambert_TFs["id"].str.split("|").str[1]
lambert_TFs

,id,ProteinSeq,DBD_coords_merged,uniprotID
0,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,MEPEGRGSLFEDSDLLHAGNPKENDVTAVLLTPGSQELMIRDMAEA...,"[[221, 243], [249, 271], [277, 299], [305, 327...",A0A087WUV0
1,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,"[[114, 182], [262, 347]]",A0AVK6
2,sp|A0PJY2|FEZF1_HUMAN Fez family zinc finger p...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,"[[260, 282], [288, 310], [316, 338], [344, 366...",A0PJY2
3,sp|A1A519|F170A_HUMAN Protein FAM170A OS=Homo ...,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,"[[1, 330]]",A1A519
4,sp|A1YPR0|ZBT7C_HUMAN Zinc finger and BTB doma...,MANDIDELIGIPFPNHSSEVLCSLNEQRHDGLLCDVLLVVQEQEYR...,"[[364, 386], [392, 414], [420, 442], [448, 469]]",A1YPR0
...,...,...,...,...
1608,sp|Q9Y6Q9|NCOA3_HUMAN Nuclear receptor coactiv...,MSGLGENLDPLASDSRKRKLPCDTPGQGLTCSGEKRRREQESKYIE...,"[[31, 83]]",Q9Y6Q9
1609,sp|Q9Y6R6|Z780B_HUMAN Zinc finger protein 780B...,MVHGSVTFRDVAIDFSQEEWECLQPDQRTLYRDVMLENYSHLISLG...,"[[165, 187], [193, 215], [221, 243], [249, 271...",Q9Y6R6
1610,sp|Q9Y6X0|SETBP_HUMAN SET-binding protein OS=H...,MESRETLSSSRQRGGESDFLPVSSAKPPAAPGCAGEPLLSTPGPGK...,"[[583, 596], [1015, 1027], [1450, 1462]]",Q9Y6X0
1611,sp|Q9Y6X8|ZHX2_HUMAN Zinc fingers and homeobox...,MASKRKSTTPCMVRTSQVVEQDVPEEVDRAKEKGIGTPQPDVAKDS...,"[[78, 101], [110, 133], [271, 317], [442, 496]...",Q9Y6X8


In [13]:
# Getting all possible gene names from uniprot 
ensembl_matches = pd.read_csv("../data/TF_uniprotIDs_to_ENST_gene.tsv", sep = "\t")
ensembl_matches["gene"] = ensembl_matches["Gene Names"].str.split(" ").str[0]
ensembl_matches

,From,Entry,Gene Names,Ensembl,gene
0,A0A087WUV0,A0A087WUV0,ZNF892,ENST00000425953.6;,ZNF892
1,A0AVK6,A0AVK6,E2F8,ENST00000250024.9;ENST00000527884.5;ENST000006...,E2F8
2,A0PJY2-1,A0PJY2,FEZF1 FEZ ZNF312B,ENST00000427185.2 [A0PJY2-2];ENST00000442488.7...,FEZF1
3,A1A519-1,A1A519,FAM170A ZNFD,ENST00000335286.10 [A1A519-2];ENST00000379555....,FAM170A
4,A1YPR0,A1YPR0,ZBTB7C APM1 ZBTB36 ZNF857C,ENST00000535628.6;ENST00000586438.5;ENST000005...,ZBTB7C
...,...,...,...,...,...
1608,Q9Y6Q9-1,Q9Y6Q9,NCOA3 AIB1 BHLHE42 RAC3 TRAM1,ENST00000371997.3 [Q9Y6Q9-3];ENST00000371998.8...,NCOA3
1609,Q9Y6R6,Q9Y6R6,ZNF780B ZNF779,ENST00000434248.6;ENST00000625684.3;ENST000006...,ZNF780B
1610,Q9Y6X0-1,Q9Y6X0,SETBP1 KIAA0437,ENST00000426838.8 [Q9Y6X0-2];ENST00000649279.2...,SETBP1
1611,Q9Y6X8,Q9Y6X8,ZHX2 AFR1 KIAA0854 RAF,ENST00000314393.6;,ZHX2


In [98]:
files = glob.glob("../data/zoonomia_toga_mca/ENST*")

# Converting codon alignment to unaligned protein sequences
for file in files:
    # Read in as dataframe  
    df = protfasta.read_fasta(file, invalid_sequence_action = 'convert') #Convert - to ""
    df = pd.DataFrame({"id" : df.keys(), "nt_seq" : df.values()})

    # Translate sequences
    df["prot_seq"] = [str(Seq(_).translate())[:-1] for _ in df["nt_seq"]]

    ENST = file.split("/")[-1].split(".")[0]
    gene =  file.split("/")[-1].split(".")[1]

    # Save protein sequences in dictionary
    prot_dict = dict(zip(df["id"], df["prot_seq"]))
    protfasta.write_fasta(prot_dict, "../data/zoonomia_toga_mca/prot_fastas/" + gene)
 

In [99]:
# Slow to run
genes = [_.split(".")[-2] for _ in files]
for gene in genes:
    os.system("mafft --auto --quiet ../data/zoonomia_toga_mca/prot_fastas/" + gene + " > ../data/zoonomia_toga_mca/prot_alignments/" + gene + ".fasta")

In [45]:
# All possible gene names
# Uniprot IDs from Lambert
# Uniprot -> all gene names possible
# Will query all
ensembl_matches["all_genes"] = ensembl_matches["Gene Names"].str.split(" ")
all_gene_names = ensembl_matches[["all_genes"]].explode("all_genes")
all_gene_names["len"] = all_gene_names["all_genes"].str.len()
all_gene_names = all_gene_names[all_gene_names["len"] > 3]
all_gene_names

,all_genes,len
0,ZNF892,6
1,E2F8,4
2,FEZF1,5
2,ZNF312B,7
3,FAM170A,7
...,...,...
1611,AFR1,4
1611,KIAA0854,8
1612,CAMTA1,6
1612,KIAA0833,8


# 2. Scraping zoonomia website to get paths to full protein alignments to donwload

In [50]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


In [51]:
base_url = "https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/"


In [52]:
response = requests.get(base_url, verify = False)

In [53]:
names = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the page content with BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find all links (anchor tags with href attributes)
    links = soup.find_all("a", href=True)

    for link in links:
        for gene in all_gene_names["all_genes"]:
            if gene in link['href']:
                # print(gene)
                if str(link['href']).split(".")[-3] == gene:
                    names.append(link['href'])
            
    # # Filter the links that contain 'ELP1' in their name
    # elp1_files = [link['href'] for link in links if "ELP1" in link['href']]

In [54]:
len(names)

1214

In [55]:
names[:3]

['ENST00000006015.HOXA11.fasta.gz',
 'ENST00000007699.YBX2.fasta.gz',
 'ENST00000008391.TFAP2D.fasta.gz']

In [60]:
# Check every valid enst.gene mapping

with open("../output/toga_commands_all_TFs.txt", "w") as file:
    for name in names:
        
        for ENST in uniprotID_ENSTs:
            command = "wget --no-check-certificate https://genome.senckenberg.de/download/TOGA/human_hg38_reference/MultipleCodonAlignments/" + name + " -P /Users/sanjanakotha/Desktop/Staller_Lab/SFARI/data/zoonomia_toga_mca_all_TFs/codon_alignments"
            file.write(command + "\n")


Ran gunzip *.gz in directory to unzip everything

# 3. Convert codon alignment to protein alignments

In [65]:
! mkdir ../data/zoonomia_toga_mca_all_TFs/prot_fastas/

In [69]:
files = glob.glob("/Users/sanjanakotha/Desktop/Staller_Lab/SFARI/data/zoonomia_toga_mca_all_TFs/codon_alignments/ENST*")

# Converting codon alignment to unaligned protein sequences
for file in files:
    # Read in as dataframe  
    df = protfasta.read_fasta(file, invalid_sequence_action = 'convert') #Convert - to ""
    df = pd.DataFrame({"id" : df.keys(), "nt_seq" : df.values()})

    # Translate sequences
    df["prot_seq"] = [str(Seq(_).translate())[:-1] for _ in df["nt_seq"]]

    ENST = file.split("/")[-1].split(".")[0]
    gene =  file.split("/")[-1].split(".")[1]

    # Save protein sequences in dictionary
    prot_dict = dict(zip(df["id"], df["prot_seq"]))
    protfasta.write_fasta(prot_dict, "../data/zoonomia_toga_mca_all_TFs/prot_fastas/" + gene)

# 3A. Run MAFFT

In [72]:
# Slow to run
genes = [_.split(".")[-2] for _ in files]
for gene in genes:
    os.system("mafft --auto --quiet ../data/zoonomia_toga_mca_all_TFs/prot_fastas/" + gene + " > ../data/zoonomia_toga_mca_all_TFs/prot_alignments/" + gene + ".fasta")